1.The way to label rpn target as follow
$$U_1=\{j\bracevert IOU(y^{(i)},a_{j})\geq t, \text{for i is gt }\}$$

$$U_2=\{argmax_{j}IOU(y^{(i)},a_{j}),\text{for i is gt }\}$$

$$Lable(i)=1 \rightarrow i \in U_1,U_2$$

$$
 lable(i) =
  \begin{cases}
    1      & \quad \text{if } i \text{ in U1 or U2}\\
    0      & \quad \text{otherwise }
  \end{cases}
$$

### 2.create anchor box

>First convert between x1y1x2y2 between wh
$$cx=\frac{x_2+x_1}{2}$$

$$cy=\frac{y_2+y_1}{2}$$

$$w=x_2-x_1+1$$

$$h=y_2-y_1+1$$ 

$$\begin{bmatrix}
   cx \\
   cy \\
   w \\
   h \\
\end{bmatrix}=
\begin{bmatrix}
\frac{1}{2} &0 & \frac{1}{2} & 0 \\
 0 & \frac{1}{2}& 0 & \frac{1}{2} \\
-1 &0 & 1& 0 \\
0 &-1 &0 & 1 \\
\end{bmatrix}*
\begin{bmatrix}
   x_1 \\
   y_1 \\
   x_2 \\
   y_2\\
\end{bmatrix}+
\begin{bmatrix}
   0 \\
   0 \\
   1 \\
   1\\
\end{bmatrix}
$$

$$
\begin{bmatrix}
   x_1 \\
   y_1 \\
   x_2 \\
   y_2\\
\end{bmatrix}=
\begin{bmatrix}
 1 &0 & -\frac{1}{2} & 0 \\
 0 & 1& 0 & -\frac{1}{2} \\
 1 &0 & \frac{1}{2}& 0 \\
 0 &1 &0 & \frac{1}{2} \\
\end{bmatrix}*
\begin{bmatrix}
   cx \\
   cy \\
   w-1 \\
   h-1\\
\end{bmatrix}
$$

In [57]:
import numpy as np
def xy2wh(boxes):
    '''
    boxes:(N,4) x1y1x2y2
    '''
    boxes=boxes.copy()
    M=np.array([[.5,0,.5,0],[0,.5,0,.5],[-1,0,1,0],[0,-1,0,1]]).T
    b=np.array([0,0,1,1])
    return boxes.dot(M)+b
def wh2xy(boxes):
    '''
    boxes:(N,4) cxcywh
    '''
    boxes=boxes.copy()
    M=np.array([[1,0,-.5,0],[0,1,0,-.5],[1,0,0.5,0],[0,1,0,0.5]]).T
    b=np.array([0,0,-1,-1])
    return (boxes+b).dot(M)
boxes=np.array([[1,1,32,32]])
print(xy2wh(boxes))
print(wh2xy(xy2wh(boxes)))

[[16.5 16.5 32.  32. ]]
[[ 1.  1. 32. 32.]]


In [58]:
def setCenter(boxes,center):
    '''
    boxes:cx,cy,w,h,(N,4)
    center:X,Y
    '''
    boxes=boxes.copy()
    cx,cy=center
    boxes[:,[0,1]]=(cx,cy)
    return boxes
boxes=np.array([[1,1,32,32]])
CCC=xy2wh(boxes)
print(CCC)
DDD=setCenter(CCC,(8,8))
print(DDD)
print(wh2xy(DDD))
# print(xy2wh(CCC))

[[16.5 16.5 32.  32. ]]
[[ 8.  8. 32. 32.]]
[[-7.5 -7.5 23.5 23.5]]


>Next make boxes with different aspect ratio and same area!

In [83]:
def mkbox(size,ratios):
    '''
        size:determine output boxes have size s*s
        ratio:list of ratio
        return:boxes(N,4) (x1,y1,x2,y2) x1,y1=0,0
    '''
    
    ret=[(0,0,size*np.sqrt(r)-1,size/np.sqrt(r)-1) for r in ratios]
    return np.array(ret)
x=mkbox(98,[.5,1,2])
x1=xy2wh(x)
print(x1[:,2]*x1[:,3])
print(x1)
print()

[9604. 9604. 9604.]
[[ 34.14823228  68.79646456  69.29646456 138.59292911]
 [ 48.5         48.5         98.          98.        ]
 [ 68.79646456  34.14823228 138.59292911  69.29646456]]



In [92]:
def createAnchorBoxes(base,scales,ratios):
    anchors=np.concatenate([mkbox(base*s,ratios) for s in scales],axis=0)
    anchors=xy2wh(anchors)
    anchors=setCenter(anchors,(base//2,base//2))
    anchors=wh2xy(anchors)
    return anchors
X=createAnchorBoxes(16,[2,4,8,16],[.5,1.,2.])
X_wh=xy2wh(X)
print('center:',X_wh[:,0:2])
print()
print('area:',X_wh[:,2]*X_wh[:,3])

center: [[8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]]

area: [ 1024.  1024.  1024.  4096.  4096.  4096. 16384. 16384. 16384. 65536.
 65536. 65536.]


In [99]:
import torch

In [112]:
a=torch.rand(128,55,4)
b=torch.rand(128,43,4)

torch.matmul(a,b.transpose(-2,-1)).shape #128,55,43

torch.Size([128, 55, 43])

In [98]:
import numpy as np
np.triu(np.ones((3,3)),0)

array([[1., 1., 1.],
       [0., 1., 1.],
       [0., 0., 1.]])

In [104]:
-1e9

-1000000000.0